## Opinion Reveiw 데이터 세트를 이용한 문서 군집화 수행하기
- 51개의 텍스트 파일
- 각 파일은 Tripadvisor(호텔), Edumunds.com(자동차), Amazon.com(전자제품) 사이트에서 가져온 리뷰 문서
- 텍스트 분류 기반의 문서 분류에서 사전에 결정 카테고리 값을 가진 학습 데이터 세트가 필요한 데 반해, 문서 군집화는 학습 데이터 세트가 필요 없는 비지도학습 기반으로 동작
- 군집화 기법을 활용해 텍스트 기반의 문서 군집화 해보자

In [8]:
import pandas as pd
import glob, os

path = 'C:/Users/영현/Desktop/OpinosisDataset1.0/topics'

# path로 저장한 디렉터리 밑에 있는 모든 .data 파일의 파일명 리스트로 취합
all_files = glob.glob(os.path.join(path, '*data'))
filename_list = []
opinion_text = []

# 개별 파일의 파일명은 filename_list로 취합
# 개별 파일의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환해 opinion_text list로 취합
for file_ in all_files:
    # 개별 파일 읽어서 DataFrame으로 생성
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
    
    # 절대 경로로 주어진 파일명 가공
    # 맨 마지막 .data 확장자도 제거
    filename_ = file_.split('\\')[-1]
    filename = filename_.split('.')[0]
    
    # 파일명 list와 파일 내용 list에 파일명과 파일 내용 추가
    filename_list.append(filename)
    opinion_text.append(df.to_string())
    
# 파일명 list와 파일 내용 list 객체를 DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,", and is very, very acc..."
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and..."
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my ...
3,battery-life_ipod_nano_8gb,short battery life I moved up from a...
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 2..."


- TF-IDF 형태로 피처 벡터화
- tokenizer는 이전 예제에서 Lemmatization을 구현한 LemNormalize()함수를 이용, ngram은 (1,2), min_df와 max_df 범위를 설정해 피처의 개수 제한

In [14]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
# ord(c)는 문자의 아스키 코드 값을 돌려주는 함수
# Punctuation: 따옴표,마침표,물음표 등의 문장부호
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                            ngram_range=(1,2), min_df=0.05, max_df=0.85)

# opinion_text 칼럼 값으로 빅처 벡터화 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

- 문서별 텍스트가 TF-IDF 변환된 피처 벡터화 행렬 데이터에 대해서 군집화를 수행해 어떤 문서끼리 군집되는지 확인해보자
- 군집화 기법은 K-평균 적용
- 문서의 유형은 전자제품, 자동차, 호텔로 돼 있으며, 전자제품은 다시 네비게이션, 아이팟, 킨들, 랩탑 컴퓨터 등과 같은 세부 요소로 나뉨
- 먼저 5개의 중심(Centroid) 기반으로 어떻게 군집화되는지 확인


In [16]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [17]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,", and is very, very acc...",3
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and...",0
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my ...,4
3,battery-life_ipod_nano_8gb,short battery life I moved up from a...,4
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 2...",4


In [18]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')
# cluster #0은 호텔에 대한 리뷰로 군집화 됨

,filename,opinion_text,cluster_label
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and...",0
32,room_holiday_inn_london,"We arrived at 23,30 hours and they could n...",0
30,rooms_bestwestern_hotel_sfo,"Great Location , Nice Rooms , Helpless...",0
31,rooms_swissotel_chicago,The Swissotel is one of our favorite hotel...,0


In [19]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')
# cluster #1은 #0과 같이 대부분 호텔에 대한 리뷰로 군집화

,filename,opinion_text,cluster_label
13,food_holiday_inn_london,The room was packed to capacity with queu...,1
14,food_swissotel_chicago,The food for our event was deli...,1
15,free_bestwestern_hotel_sfo,The wine reception is a great idea as it i...,1
20,location_bestwestern_hotel_sfo,"Good Value good location , ideal ...",1
21,location_holiday_inn_london,Great location for tube and we crammed in...,1
24,parking_bestwestern_hotel_sfo,Parking was expensive but I think this is ...,1
28,price_holiday_inn_london,"All in all, a normal chain hotel on a nice...",1
38,service_bestwestern_hotel_sfo,Both of us having worked in tourism for o...,1
39,service_holiday_inn_london,"not customer, oriented hotelvery low servi...",1
40,service_swissotel_hotel_chicago,Mediocre room and service for a very extr...,1


In [20]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')
# cluster #3는 토요타(Toyota)와 혼다(Honda) 등의 자동차에 대한 리뷰로 군집화

,filename,opinion_text,cluster_label
6,comfort_honda_accord_2008,"Drivers seat not comfortable, the car its...",2
7,comfort_toyota_camry_2007,Ride seems comfortable and gas mileage fa...,2
16,gas_mileage_toyota_camry_2007,Ride seems comfortable and gas mileage fa...,2
17,interior_honda_accord_2008,I love the new body style and the interior...,2
18,interior_toyota_camry_2007,"First of all, the interior has way too ma...",2
22,mileage_honda_accord_2008,"It's quiet, get good gas mileage and look...",2
25,performance_honda_accord_2008,"Very happy with my 08 Accord, performance i...",2
29,quality_toyota_camry_2007,I previously owned a Toyota 4Runner which ...,2
37,seats_honda_accord_2008,Front seats are very uncomfor...,2
47,transmission_toyota_camry_2007,"After slowing down, transmission has to b...",2


In [21]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')
# cluster #4는 킨들, 아이팟, 넷북 등의 포터블 전자기기에 대한 리뷰로 군집화

,filename,opinion_text,cluster_label
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my ...,4
3,battery-life_ipod_nano_8gb,short battery life I moved up from a...,4
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 2...",4
11,features_windows7,"I had to uninstall anti, virus and selecte...",4
19,keyboard_netbook_1005ha,", I think the new keyboard rivals the gre...",4
26,performance_netbook_1005ha,The Eee Super Hybrid Engine utility lets u...,4
34,screen_garmin_nuvi_255W_gps,It is easy to read and when touching the...,4
35,screen_ipod_nano_8gb,"As always, the video screen is sharp and b...",4
36,screen_netbook_1005ha,Keep in mind that once you get in a room ...,4
41,size_asus_netbook_1005ha,A few other things I'd like to point out i...,4


- 전반적으로 군집화된 결과를 살펴보면 군집 개수가 약간 많게 설정돼 있어서 세분화되어 군집화된 경향 있음
- 중심 개수를 5개에서 3개로 낮춰서 3개 그룹으로 군집화한 뒤 결과를 확인

In [24]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

# cluster #0: 포터블 전자기기 리뷰
# cluster #1: 호텔 리뷰
# cluster #2: 자동차 리뷰

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,", and is very, very acc...",0
48,updates_garmin_nuvi_255W_gps,Another thing to consider was that I paid $...,0
47,transmission_toyota_camry_2007,"After slowing down, transmission has to b...",0
44,speed_windows7,"Windows 7 is quite simply faster, more sta...",0
43,speed_garmin_nuvi_255W_gps,Another feature on the 255w is a display of...,0
42,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,0
41,size_asus_netbook_1005ha,A few other things I'd like to point out i...,0
36,screen_netbook_1005ha,Keep in mind that once you get in a room ...,0
35,screen_ipod_nano_8gb,"As always, the video screen is sharp and b...",0
34,screen_garmin_nuvi_255W_gps,It is easy to read and when touching the...,0


## 군집별 핵심 단어 추출하기
- 각 군집을 구성하는 핵심 단어가 어떤 것이 있는지 확인해보자
- KMeans 객체는 각 군집을 구성하는 단어 피처가 군집의 중심(Centriod)을 기준으로 얼마나 가깝게 위치해 있는지 cluster_centers_ 속성으로 제공
    - 배열 값으로 제공되며, 행은 개별 군집을, 열은 개별 피처를 의미
    - 각 배열 내의 값은 개별 군집 내의 상대 위치를 숫자 값으로 표현한 일종의 좌표 값
    - ex) cluster_centers[0,1]은 0번 군집에서 두 번째 피처의 위치 값

In [29]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape:', cluster_centers.shape)
print(cluster_centers)

# 군집이 3개, word 피처가 2409개
# 각 행의 배열 값은 각 군집 내의 2409개 피처의 위치가 개별 중심과 얼마나 가까운가를 상대 값으로 나타낸 것
# 0~1사이 값, 1에 가까울수록 중심과 가까운 값 의미

cluster_centers shape: (3, 2409)
[[0.0174987  0.         0.         ... 0.         0.         0.00452955]
 [0.         0.00170335 0.0025537  ... 0.0032582  0.00349413 0.        ]
 [0.         0.00152566 0.         ... 0.         0.         0.        ]]


- cluster_centers_ 속성값을 이용해 각 군집별 핵심 단어 찾아보자
- cluster_centers_ 속성은 넘파이의 ndarray
- ndarray의 argsort[:,::-1]를 이용하면 cluster_centers 배열 내 값이 큰 순으로 정렬된 위치 인덱스 값 반환

In [47]:
# 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 반환
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num,
                       top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array의 값이 큰 순으로 정렬된 인덱스 값 반환
    # 군집 중심점(Centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    # 개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # cluster_centers_.argsort()[:,::-1]로 구한 인덱스를 이용해 top n 피처 단어를 구함
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [feature_names[ind] for ind in top_feature_indexes]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details

In [50]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('###### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reveiws 파일명:', cluster_detail['filenames'][:7])
        print('=============================================')

In [51]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,
                                     feature_names=feature_names, clusters_num=3, top_n_features=10)
print_cluster_details(cluster_details)

###### Cluster 0
Top features: ['screen', 'battery', 'life', 'battery life', 'keyboard', 'kindle', 'size', 'button', 'easy', 'voice']
Reveiws 파일명: ['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps']
###### Cluster 1
Top features: ['room', 'hotel', 'service', 'location', 'staff', 'food', 'clean', 'bathroom', 'parking', 'room wa']
Reveiws 파일명: ['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwestern_hotel_sfo', 'location_bestwestern_hotel_sfo', 'location_holiday_inn_london', 'parking_bestwestern_hotel_sfo']
###### Cluster 2
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'car', 'gas', 'gas mileage', 'comfort', 'ride', 'performance']
Reveiws 파일명: ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_honda_accord_2008',